In [1]:
from source.chroma_manager import ChromaDBManager, CustomEmbeddingFunction
from source.llm_manager import LMStudioClient
from prompt.prompts import QuestionsToDoc
from tqdm import tqdm
import ast
import json
import logging

DB_CONFIG = LMStudioClient.load_config("config/embedding/questions_gen.yaml")
MODEL_CONFIG_PATH = "config/model_question_gen.yaml"

logging.basicConfig(filename="log/question_gen.log", level=logging.ERROR, format="%(asctime)s - %(levelname)s - %(message)s")


# Инициализация менеджеров
source_chroma = ChromaDBManager(
    storage_path=DB_CONFIG["chroma_path"],
    collection_name=DB_CONFIG["collection_name"],
    embedding_function=CustomEmbeddingFunction(DB_CONFIG["model_name"])
)

llm = LMStudioClient(MODEL_CONFIG_PATH)  # URL настроить под свой сервер

# Извлечение всех документов
all_documents = source_chroma.collection.get()
doc_ids = all_documents.get("ids", [])
doc_texts = all_documents.get("documents", [])
doc_metadata = all_documents.get("metadatas", [])

C:\Users\Igorexy\AppData\Roaming\Python\Python312\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


e:\ImportantFiles\Documents\University\Magic App\source\llm_manager.py:9: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(


In [2]:
len(doc_ids)

549

In [3]:
# doc_ids_batch = doc_ids[300:]
# doc_texts_batch = doc_texts[300:]
# doc_metadata_batch = doc_metadata[300:]

In [4]:
def safe_str_to_dict(s: str):
    """
    Преобразует строку в словарь, если возможно.
    В противном случае возвращает False.
    """
    try:
        result = ast.literal_eval(s)
        if isinstance(result, dict):
            return result
        return False
    except (ValueError, SyntaxError):
        return False

In [5]:
# Список ids где в метадате отсутвуют вопросы
ids_to_generate = []
for ids, metadatas in zip(doc_ids, doc_metadata):
    if 'questions' not in list(metadatas.keys()):
        ids_to_generate.append(ids)
len(ids_to_generate)

0

In [5]:
for doc_id, doc_text, metadata in tqdm(zip(doc_ids, doc_texts, doc_metadata)):
    if doc_id in ids_to_generate:
        # Генерация вопросов через LLM
        prompt = QuestionsToDoc.QUESTION_FORMATION.format(document=doc_text)
        # print(repr(prompt))  # Выведет строку в raw-формате
        questions = llm.post_completion(user_input=prompt)
        llm.clear_context() # очищаем контекст
        questions_dict = safe_str_to_dict(questions)
        if questions_dict == False:
            prompt = QuestionsToDoc.CORRECTING_DICKTIONARY.format(input=questions)
            questions = llm.post_completion(user_input=prompt)
            llm.clear_context() # очищаем контекст
            questions_dict = safe_str_to_dict(questions)
            if questions_dict == False:
                print("Словарь не сгенерирован")
                print(questions)
                logging.error(f" doc_id={doc_id}: questions_dict={questions}")
                continue
            
        # Обновление метаданных
        # print("Успешно")
        # print(questions_dict)
        metadata["questions"] = json.dumps(questions_dict)
        source_chroma.update_document(document_id=doc_id, new_text=doc_text, new_metadata=metadata)

21it [11:07, 34.07s/it]<unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:5: SyntaxWarning: invalid escape sequence '\('
28it [15:38, 44.38s/it]<unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:5: SyntaxWarning: invalid escape sequence '\('
29it [16:26, 45.70s/it]<unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:5: SyntaxWarning: invalid escape sequence '\('
134it [1:08:51, 26.00s/it]<unknown>:5: SyntaxWarning: invalid escape sequence '\('
145it [1:31:32, 52.81s/it] <unknown>:5: SyntaxWarning: invalid escape sequence '\('
152it [1:35:32, 35.53s/it]<unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:5: SyntaxWarning: invalid escape sequence '\('
154it [1:36:08, 27.08s/it]<unknown>:5: SyntaxWarning: invalid escape sequence '\('
157it [1:38:07, 35.40s/it]<unknown>:2: SyntaxWarning: invalid escape sequence '\('
<unknown>:3: SyntaxWarning: invalid escape sequence '\('
<unknown>:4: SyntaxWarning: invalid escape sequence '\('
<u